# Downloading dataset

I have downloaded the dataset manually and uploaded it in the input section of kaggle notebook.

Dataset can be downloaded here: https://storage.googleapis.com/wandb_datasets/nature_12K.zip

# Part A: Training from scratch

## Question 1

### computing the shape of input images, so that we can design CNN accordingly

In [2]:
from PIL import Image
import numpy as np

img_1 = Image.open('/kaggle/input/nature-12k/inaturalist_12K/train/Amphibia/00b5ef2b46292f2e37a62649dfaff44c.jpg')

img_array = np.array(img_1)

print(f'shape of input image is {img_array.shape}')

shape of input image is (600, 800, 3)


### Building 5 layer CNN model 

In [4]:
# Importing libraries

import torch
import torch.nn as nn
import torch.nn.functional as F

In [21]:
# Building a class for CNN model

class CNN(nn.Module):
    def __init__(
        self,
        input_size = (600, 800, 3), # getting this shape from previous computation
        num_classes = 20,
        conv_channels = [16,32,64,128,256],
        kernel_sizes = [3,3,3,3,3],
        activation_fn = nn.ReLU,
        dense_neurons = 521 , 
        max_pool_size = 2):

        super(CNN,self).__init__()


        self.conv_layers = nn.ModuleList()
        in_channels = input_size[2]

        # Building the convolutional layers
        for out_channels, kernel_size in zip(conv_channels, kernel_sizes):
            self.conv_layers.append(
                nn.Sequential(
                    nn.Conv2d(
                        in_channels,
                        out_channels,
                        kernel_size = kernel_size,
                        padding = kernel_size // 2
                    ),
                    activation_fn(),
                    nn.MaxPool2d(kernel_size = max_pool_size)
                )
            )
            in_channels = out_channels

        # Calculating final feature map size to build the dense neural network
        height, width = input_size[0], input_size[1]

        for _ in range(len(conv_channels)):
            height //= max_pool_size
            width //= max_pool_size

        flatten_size = conv_channels[-1] * height* width

        # Building dense layer
        self.fc1 = nn.Linear(flatten_size, dense_neurons)

        # Building output layer
        self.fc_out = nn.Linear(dense_neurons, num_classes)


    # defingn the forward function
    def forward(self,x):
        
        # passing input to each layer
        for layer in self.conv_layers:
            x = layer(x)

         # flatten the layer
        x = x.view(x.size(0),-1)

        x = F.relu(self.fc1(x))

        # passing to the output layer
        x = self.fc_out(x)

        return x
        
            

In [25]:
model = CNN()

print(model)

CNN(
  (conv_layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (4): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPo

## Question 2